## Import

In [2]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from glob import glob
from tqdm import tqdm

### bbox area가 100000 이상인 객체 확인

In [ ]:
# 언어별 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']
bbox_sizes_per_language = {}

# 각 언어별로 bounding box 크기 수집 및 바운딩 박스 그리기
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')
    bbox_sizes = []

    # JSON 파일 확인
    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 bounding box 크기 추출
    with open(json_path, 'r') as f:
        data = json.load(f)

        if not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        # 이미지 키를 사용해 words 접근
        for image_key, image_data in data['images'].items():
            if 'words' not in image_data:
                print(f"Error: 'words' key not found in {json_path} for image {image_key}")
                continue
            
            # 이미지를 로드
            image_path = os.path.join(data_dir, lang, 'img/train', image_key)
            image = Image.open(image_path)
            plt.figure(figsize=(10, 10))
            plt.imshow(image)

            # Bounding box 크기 계산 및 바운딩 박스 그리기
            area_exceeds_threshold = False  # 바운딩 박스 크기가 임계값을 초과하는지 확인
            for item in image_data['words'].values():
                points = item.get('points')
                if not points:
                    print(f"Warning: 'points' key missing in item {item}")
                    continue
                
                x_min, y_min = min([p[0] for p in points]), min([p[1] for p in points])
                x_max, y_max = max([p[0] for p in points]), max([p[1] for p in points])
                width = x_max - x_min
                height = y_max - y_min
                area = width * height
                
                # 바운딩 박스 사이즈가 100,000이 넘는 경우에만 그리기
                if area > 100000:
                    plt.gca().add_patch(plt.Rectangle((x_min, y_min), width, height, 
                                                        edgecolor='red', facecolor='none', linewidth=2))
                    area_exceeds_threshold = True  # 조건을 만족하는 바운딩 박스가 있음을 표시

            # 바운딩 박스가 하나라도 그려졌을 때만 이미지 출력
            if area_exceeds_threshold:
                plt.title(f'Bounding Boxes for {lang} - {image_key}')
                plt.axis('off')  # 축 숨기기
                plt.show()  # 이미지를 출력
            else:
                plt.close()  # 바운딩 박스가 없으면 이미지를 닫음

    bbox_sizes_per_language[lang] = bbox_sizes


## random으로 이미지 출력

In [ ]:
import json
import os
import random
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']

# 랜덤 이미지 출력 함수
def display_random_images_with_bboxes(data_dir, languages, num_images=10):
    all_images = []

    # 언어별 JSON 파일 읽기
    for lang in languages:
        json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')

        # JSON 파일 확인
        if not os.path.exists(json_path):
            print(f"Error: JSON file not found at {json_path}")
            continue

        # JSON 파일에서 이미지 키 수집
        with open(json_path, 'r') as f:
            data = json.load(f)
            if 'images' in data:
                all_images.extend(data['images'].keys())

    # 랜덤으로 이미지 선택
    if all_images:
        random_image_keys = random.sample(all_images, min(num_images, len(all_images)))
        
        for random_image_key in random_image_keys:
            print(f"Selected Image Key: {random_image_key}")

            # 언어별로 이미지 경로를 생성하고 파일이 존재하는지 확인
            for lang in languages:
                image_path = os.path.join(data_dir, lang, 'img', 'train', random_image_key)
                print(f"Checking for image at: {image_path}")  # 경로 출력
                
                # 이미지 파일 확인
                if os.path.exists(image_path):
                    image = Image.open(image_path).convert("RGB")
                    draw = ImageDraw.Draw(image)

                    # 바운딩 박스 그리기
                    if 'images' in data and random_image_key in data['images']:
                        words_info = data['images'][random_image_key].get('words', {})
                        for word_info in words_info.values():
                            if 'points' in word_info:
                                # 좌표를 (x, y) 형식의 튜플로 변환
                                bbox = [(point[0], point[1]) for point in word_info['points']]
                                draw.polygon(bbox, outline="red", width=2)

                    # 이미지 출력
                    plt.figure(figsize=(10, 10))
                    plt.imshow(image)
                    plt.title(f"Randomly Selected Image: {random_image_key}")
                    plt.axis("off")
                    plt.show()
                    break  # 이미지를 찾았으면 다음 이미지로 넘어감
                else:
                    print(f"Error: Image file not found at {image_path}")

    else:
        print("No images found in any JSON files.")

# 랜덤 이미지 출력 (10개)와 바운딩 박스 그리기
display_random_images_with_bboxes(data_dir, languages, num_images=10)


## bounding box의 위치 분석

In [ ]:
import json
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np

# 언어별 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']

# 각 언어별 바운딩 박스 중심 좌표 저장
bbox_centers_per_language = {lang: [] for lang in languages}

# 각 언어별로 bounding box 크기 수집 및 바운딩 박스 중심 좌표 계산
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')

    # JSON 파일 확인
    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 bounding box 중심 좌표 추출
    with open(json_path, 'r') as f:
        data = json.load(f)

        if not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        # 이미지 키를 사용해 words 접근
        for image_key, image_data in data['images'].items():
            if 'words' not in image_data:
                print(f"Error: 'words' key not found in {json_path} for image {image_key}")
                continue
            
            for item in image_data['words'].values():
                points = item.get('points')
                if not points:
                    print(f"Warning: 'points' key missing in item {item}")
                    continue
                
                x_min, y_min = min([p[0] for p in points]), min([p[1] for p in points])
                x_max, y_max = max([p[0] for p in points]), max([p[1] for p in points])
                
                # 바운딩 박스의 중심 좌표 계산
                center_x = (x_min + x_max) / 2
                center_y = (y_min + y_max) / 2
                bbox_centers_per_language[lang].append((center_x, center_y))

# 히스토그램 시각화
for lang, centers in bbox_centers_per_language.items():
    if not centers:
        continue

    centers = np.array(centers)
    plt.figure(figsize=(10, 6))
    plt.hist2d(centers[:, 0], centers[:, 1], bins=50, cmap='Blues', density=True)
    plt.colorbar(label='Density')
    plt.title(f'Bounding Box Center Distribution for {lang}')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.axis('equal')  # 비율을 유지
    plt.show()


## 언어별 텍스트 길이 분포 확인

In [ ]:
import json
import matplotlib.pyplot as plt
import os

# 언어별 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']

# 각 언어별 텍스트 길이 수집
text_lengths = {lang: [] for lang in languages}

# 각 언어별로 텍스트 길이 수집
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')

    # JSON 파일 확인
    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 텍스트 길이 추출
    with open(json_path, 'r') as f:
        data = json.load(f)

        if not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        # 이미지 키를 사용해 words 접근
        for image_data in data['images'].values():
            if 'words' not in image_data:
                continue
            
            for item in image_data['words'].values():
                # 텍스트 길이 수집
                text = item.get('transcription', '')
                if text:  # text가 None이 아니고 비어있지 않은 경우에만 길이를 추가
                    text_lengths[lang].append(len(text))

# 언어별 텍스트 길이 분포 시각화
for lang in languages:
    plt.figure(figsize=(10, 6))
    plt.hist(text_lengths[lang], bins=30, alpha=0.7, color='blue', edgecolor='black')
    plt.title(f'Text Length Distribution for {lang}')
    plt.xlabel('Length of Text')
    plt.ylabel('Frequency')
    plt.grid()
    plt.show()


## 이미지당 면적 대비 bounding box 면적 합의 비율 분포

In [ ]:
# 언어별 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']
area_ratios_per_language = {lang: [] for lang in languages}  # 비율을 저장할 딕셔너리

# 각 언어별로 이미지 면적 대비 객체 면적 비율 분석
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')

    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 bounding box 크기 추출
    with open(json_path, 'r') as f:
        data = json.load(f)

        if not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        for image_key, image_data in data['images'].items():
            if 'words' not in image_data:
                continue
            
            # 이미지를 로드
            image_path = os.path.join(data_dir, lang, 'img/train', image_key)
            image = Image.open(image_path)
            image_width, image_height = image.size
            image_area = image_width * image_height  # 이미지 면적 계산
            
            total_bbox_area = 0  # 모든 바운딩 박스의 면적 합

            # 각 객체의 면적 계산
            for item in image_data['words'].values():
                points = item.get('points')
                if not points:
                    continue

                # 바운딩 박스의 꼭짓점 좌표
                x_coords = [p[0] for p in points]
                y_coords = [p[1] for p in points]

                # 면적 계산
                bbox_width = max(x_coords) - min(x_coords)
                bbox_height = max(y_coords) - min(y_coords)
                bbox_area = bbox_width * bbox_height
                
                total_bbox_area += bbox_area  # 면적 합산

            # 면적 비율 계산
            area_ratio = total_bbox_area / image_area if image_area > 0 else 0
            area_ratios_per_language[lang].append(area_ratio)

# 결과 시각화
for lang, ratios in area_ratios_per_language.items():
    plt.figure(figsize=(10, 5))
    plt.hist(ratios, bins=30, alpha=0.7, label=lang)
    plt.title(f'Area Ratios of Bounding Boxes to Image Area for {lang}')
    plt.xlabel('Area Ratio')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid()
    plt.show()


## noise와 background 분석

In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 언어별 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']

# 평균 밝기 및 대비 계산을 위한 리스트
brightness_values = []
contrast_values = []

# 전체 이미지에 대한 밝기 및 대비 계산
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')
    
    # JSON 파일 확인
    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 이미지 정보 추출
    with open(json_path, 'r') as f:
        data = json.load(f)

        if not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        # 이미지 키를 사용해 반복
        for image_key, image_data in data['images'].items():
            # 이미지를 로드
            image_path = os.path.join(data_dir, lang, 'img/train', image_key)
            image = Image.open(image_path).convert('L')  # 그레이스케일로 변환
            
            # 평균 밝기 계산
            brightness = np.mean(image)
            brightness_values.append(brightness)
            
            # 대비 계산 (표준편차)
            contrast = np.std(np.array(image))
            contrast_values.append(contrast)

# 평균 밝기 및 대비 시각화
plt.figure(figsize=(15, 5))

# 밝기 분포
plt.subplot(1, 2, 1)
plt.hist(brightness_values, bins=30, color='skyblue', alpha=0.7)
plt.title('Brightness Distribution')
plt.xlabel('Brightness')
plt.ylabel('Frequency')

# 대비 분포
plt.subplot(1, 2, 2)
plt.hist(contrast_values, bins=30, color='salmon', alpha=0.7)
plt.title('Contrast Distribution')
plt.xlabel('Contrast')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# 전체 평균 밝기 및 대비
average_brightness = np.mean(brightness_values)
average_contrast = np.mean(contrast_values)

print(f'Average Brightness: {average_brightness:.2f}')
print(f'Average Contrast: {average_contrast:.2f}')


In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 언어별 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']

# 평균 밝기 및 대비 계산을 위한 리스트
brightness_values = []
contrast_values = []
image_keys = []  # 이미지 키를 저장할 리스트

# 전체 이미지에 대한 밝기 및 대비 계산
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')
    
    # JSON 파일 확인
    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 이미지 정보 추출
    with open(json_path, 'r') as f:
        data = json.load(f)

        if not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        # 이미지 키를 사용해 반복
        for image_key, image_data in data['images'].items():
            # 이미지를 로드
            image_path = os.path.join(data_dir, lang, 'img/train', image_key)
            image = Image.open(image_path).convert('L')  # 그레이스케일로 변환
            
            # 평균 밝기 계산
            brightness = np.mean(image)
            brightness_values.append(brightness)
            
            # 대비 계산 (표준편차)
            contrast = np.std(np.array(image))
            contrast_values.append(contrast)

            # 이미지 키 저장
            image_keys.append((lang, image_key))  # 언어와 이미지 키 저장

# 평균 및 표준편차 계산
mean_brightness = np.mean(brightness_values)
std_brightness = np.std(brightness_values)
mean_contrast = np.mean(contrast_values)
std_contrast = np.std(contrast_values)

# 극단적인 밝기 및 대비 값 식별
extreme_brightness_indices = [i for i, b in enumerate(brightness_values) if b < (mean_brightness - 2 * std_brightness) or b > (mean_brightness + 2 * std_brightness)]
extreme_contrast_indices = [i for i, c in enumerate(contrast_values) if c < (mean_contrast - 2 * std_contrast) or c > (mean_contrast + 2 * std_contrast)]

# 극단적인 이미지 경로 저장
extreme_images = []
unique_indices = set(extreme_brightness_indices + extreme_contrast_indices)  # 중복 인덱스 제거

for index in unique_indices:
    lang, image_key = image_keys[index]  # 언어와 이미지 키 가져오기
    image_path = os.path.join(data_dir, lang, 'img/train', image_key)
    extreme_images.append(image_path)

# 극단적인 이미지 시각화
plt.figure(figsize=(15, 15))
for i, img_path in enumerate(extreme_images):
    plt.subplot(len(extreme_images) // 3 + 1, 3, i + 1)
    img = Image.open(img_path)
    plt.imshow(img, cmap='gray')
    plt.title(f'Extreme Image {i + 1}')
    plt.axis('off')

plt.tight_layout()
plt.show()

# 전체 평균 밝기 및 대비 출력
print(f'Average Brightness: {mean_brightness:.2f}')
print(f'Average Contrast: {mean_contrast:.2f}')


## 특수 문자 및 빈도수 분석

In [ ]:
import json
import os
from collections import Counter
import matplotlib.pyplot as plt

# 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']
special_characters = set('!@#$%^&*()_+-=[]{};:\\\'",.<>?/\\|`~')  # 분석할 특수 문자 세트

# 전체 특수 문자 빈도수 저장 Counter
total_special_char_count = Counter()

# 모든 언어의 특수 문자 빈도수 계산
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')

    # JSON 파일 확인
    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 텍스트 수집 및 특수 문자 빈도수 계산
    with open(json_path, 'r') as f:
        data = json.load(f)

        if not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        # 이미지 키를 사용해 words 접근
        for image_data in data['images'].values():
            if 'words' not in image_data:
                continue

            # 각 단어에서 특수 문자 빈도수 계산
            for item in image_data['words'].values():
                text = item.get('transcription')
                if text:  # text가 None이 아닐 경우에만
                    total_special_char_count.update(char for char in text if char in special_characters)

# 전체 빈도수 결과 출력
if total_special_char_count:
    print("\nOverall special character frequency:")
    for char, count in total_special_char_count.most_common():
        print(f"{char}: {count}")

    # 전체 그래프 그리기
    plt.figure(figsize=(10, 6))
    plt.bar(total_special_char_count.keys(), total_special_char_count.values(), color='skyblue')
    plt.title('Overall Special Character Frequency')
    plt.xlabel('Special Characters')
    plt.ylabel('Frequency')
    plt.xticks()
    plt.tight_layout()
    plt.show()
else:
    print("No special characters found in the dataset.")


In [ ]:
import json
import os
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']
special_characters = set('!@#$%^&*()_+-=[]{};:\\\'",.<>?/\\|`~')  # 분석할 특수 문자 세트

# 특수 문자가 10개 이상 포함된 객체 바운딩 박스를 그릴 이미지 목록
images_with_special_char_objects = []

# 언어별로 특수 문자가 10개 이상 포함된 객체가 있는 이미지 찾기
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')

    # JSON 파일 확인
    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 텍스트 수집 및 특수 문자 수 계산
    with open(json_path, 'r') as f:
        data = json.load(f)

        if 'images' not in data or not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        # 이미지 키를 사용해 words 접근
        for image_key, image_data in data['images'].items():
            if 'words' not in image_data:
                continue

            # 각 객체에서 특수 문자 수를 계산하여 10개 이상일 경우 저장
            special_char_boxes = []
            for word_info in image_data['words'].values():
                text = word_info.get('transcription')
                if text:
                    special_char_count = sum(1 for char in text if char in special_characters)
                    if special_char_count >= 10:
                        # 좌표를 (x, y) 형식의 튜플로 변환
                        bbox = [(point[0], point[1]) for point in word_info['points']]
                        special_char_boxes.append(bbox)

            # 특수 문자가 10개 이상인 바운딩 박스가 있는 경우 저장
            if special_char_boxes:
                images_with_special_char_objects.append((lang, image_key, special_char_boxes))

# 특수 문자가 10개 이상 포함된 객체의 바운딩 박스 그리기
for lang, image_key, special_char_boxes in images_with_special_char_objects:
    image_path = os.path.join(data_dir, lang, 'img/train', image_key)
    
    # 이미지 파일 확인
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}")
        continue

    image = Image.open(image_path).convert("RGB")

    # Draw 바운딩 박스
    draw = ImageDraw.Draw(image)
    for bbox in special_char_boxes:
        # 바운딩 박스가 다각형으로 제대로 그려지도록 튜플 형태로 전달
        draw.polygon(bbox, outline="red", width=2)

    # 이미지 출력
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title(f"Image: {image_key} with Objects containing 10+ Special Characters")
    plt.axis("off")
    plt.show()


In [ ]:
import json
import os
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# 파일 경로 설정
data_dir = '../data'
languages = ['chinese_receipt', 'japanese_receipt', 'thai_receipt', 'vietnamese_receipt']
special_characters = set('!@#$%^&*()_+-=[]{};:\\\'",.<>?/\\|`~')  # 분석할 특수 문자 세트

# 특수 문자로만 이루어진 객체 바운딩 박스를 그릴 이미지 목록
images_with_special_char_objects = []

# 언어별로 특수 문자로만 이루어진 객체가 있는 이미지 찾기
for lang in languages:
    json_path = os.path.join(data_dir, lang, 'ufo', 'train.json')

    # JSON 파일 확인
    if not os.path.exists(json_path):
        print(f"Error: File not found at {json_path}")
        continue

    # JSON 파일에서 텍스트 수집 및 특수 문자 수 계산
    with open(json_path, 'r') as f:
        data = json.load(f)

        if 'images' not in data or not data['images']:
            print(f"Error: No images found in {json_path}")
            continue

        # 이미지 키를 사용해 words 접근
        for image_key, image_data in data['images'].items():
            if 'words' not in image_data:
                continue

            # 각 객체에서 특수 문자로만 이루어진 경우 저장
            special_char_boxes = []
            for word_info in image_data['words'].values():
                text = word_info.get('transcription')
                if text and len(text) > 1 and all(char in special_characters for char in text):
                    # 좌표를 (x, y) 형식의 튜플로 변환
                    bbox = [(point[0], point[1]) for point in word_info['points']]
                    special_char_boxes.append(bbox)

            # 특수 문자로만 이루어진 바운딩 박스가 있는 경우 저장
            if special_char_boxes:
                images_with_special_char_objects.append((lang, image_key, special_char_boxes))

# 특수 문자로만 이루어진 객체의 바운딩 박스 그리기
for lang, image_key, special_char_boxes in images_with_special_char_objects:
    image_path = os.path.join(data_dir, lang, 'img/train', image_key)
    
    # 이미지 파일 확인
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}")
        continue

    image = Image.open(image_path).convert("RGB")

    # Draw 바운딩 박스
    draw = ImageDraw.Draw(image)
    for bbox in special_char_boxes:
        # 바운딩 박스가 다각형으로 제대로 그려지도록 튜플 형태로 전달
        draw.polygon(bbox, outline="red", width=2)

    # 이미지 출력
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title(f"Image: {image_key} with Objects containing Only Special Characters (Length > 1)")
    plt.axis("off")
    plt.show()
